<a href="https://colab.research.google.com/github/LayonGabriel/ProvaAP1-Classifica-oDeMotos/blob/main/ProvaAP1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Célula 1: Montar o Google Drive e instalar bibliotecas
from google.colab import drive
import os

# Monta o Google Drive no ambiente do Colab
drive.mount('/content/drive')

# Instala as bibliotecas necessárias
# 'ultralytics' para o modelo YOLOv8
# 'webcolors' para ajudar na nomeação das cores
!pip install ultralytics opencv-python-headless scikit-learn webcolors -q

print("Ambiente configurado com sucesso!")

In [ ]:
# Célula 2: Definir e criar as pastas do projeto
import os
import shutil

# Define o caminho base do projeto no seu Google Drive
caminho_base = '/content/drive/MyDrive/analise_de_imagens'

# Define a pasta de entrada (onde você deve colocar suas imagens)
pasta_entrada = os.path.join(caminho_base, 'imagens_para_analisar')

# Define a pasta de saída para as imagens classificadas
pasta_saida = os.path.join(caminho_base, 'imagens_classificadas')

# Cria as pastas se elas não existirem
os.makedirs(pasta_entrada, exist_ok=True)
os.makedirs(os.path.join(pasta_saida, 'moto'), exist_ok=True)
os.makedirs(os.path.join(pasta_saida, 'nao_moto'), exist_ok=True)

print(f"Estrutura de pastas criada em: {caminho_base}")
print(f"Por favor, adicione suas imagens na pasta: {pasta_entrada}")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Célula 3: Classificação Moto vs. Não Moto com YOLOv8
from ultralytics import YOLO
import cv2
import os
import shutil

# Carrega o modelo YOLOv8 pré-treinado
# 'yolov8n.pt' é um modelo pequeno e rápido, ideal para começar
model = YOLO('yolov8n.pt')

# Define o caminho completo da pasta de entrada
caminho_pasta_entrada_completo = '/content/drive/MyDrive/analise_de_imagens/imagens_para_analisar'

# Lista de arquivos na pasta de entrada
arquivos_imagem = [f for f in os.listdir(caminho_pasta_entrada_completo) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Define o caminho base do projeto no seu Google Drive
caminho_base = '/content/drive/MyDrive/analise_de_imagens'

# Define a pasta de saída para as imagens classificadas
pasta_saida = os.path.join(caminho_base, 'imagens_classificadas')


# Pasta temporária para motos que precisam de análise de cor
pasta_moto_temp = os.path.join(pasta_saida, 'moto_temp')
os.makedirs(pasta_moto_temp, exist_ok=True)

# Pastas de destino final
pasta_nao_moto_final = os.path.join(pasta_saida, 'nao_moto')

for nome_arquivo in arquivos_imagem:
    caminho_arquivo = os.path.join(caminho_pasta_entrada_completo, nome_arquivo)
    print(f"Processando: {nome_arquivo}...")

    # Realiza a predição na imagem
    results = model(caminho_arquivo, verbose=False)

    moto_detectada = False
    for r in results:
        for c in r.boxes.cls:
            # O ID da classe 'motorcycle' no dataset COCO é 3
            if model.names[int(c)] == 'motorcycle':
                moto_detectada = True
                break
        if moto_detectada:
            break

    # Move o arquivo para a pasta correspondente
    if moto_detectada:
        print(f"--> Moto detectada em {nome_arquivo}. Movendo para análise de cor.")
        shutil.move(caminho_arquivo, os.path.join(pasta_moto_temp, nome_arquivo))
    else:
        print(f"--> Nenhuma moto detectada em {nome_arquivo}. Movendo para 'nao_moto'.")
        shutil.move(caminho_arquivo, os.path.join(pasta_nao_moto_final, nome_arquivo))

print("\nClassificação primária (Moto vs. Não Moto) concluída!")

In [ ]:
# Célula 4: Análise de Cor e Organização Final
from sklearn.cluster import KMeans
import numpy as np
from scipy.spatial import distance
import os
import shutil
import cv2
from ultralytics import YOLO

# Carrega o modelo YOLOv8 pré-treinado
model = YOLO('yolov8n.pt')


# Dicionário de cores principais para mapeamento
# As cores estão no formato RGB (Red, Green, Blue)
CORES_PRINCIPAIS = {
    "preto": (0, 0, 0),
    "branco": (255, 255, 255),
    "cinza": (128, 128, 128),
    "vermelho": (255, 0, 0),
    "verde": (0, 128, 0),
    "azul": (0, 0, 255),
    "amarelo": (255, 255, 0),
    "laranja": (255, 165, 0),
    "prata": (192, 192, 192),
}

def get_color_name(rgb_tuple):
    min_dist = float('inf')
    closest_color = None
    for color_name, color_rgb in CORES_PRINCIPAIS.items():
        dist = distance.euclidean(rgb_tuple, color_rgb)
        if dist < min_dist:
            min_dist = dist
            closest_color = color_name
    return closest_color

# Define o caminho base do projeto no seu Google Drive
caminho_base = '/content/drive/MyDrive/analise_de_imagens'

# Define a pasta de saída para as imagens classificadas
pasta_saida = os.path.join(caminho_base, 'imagens_classificadas')

# Pasta temporária para motos que precisam de análise de cor
pasta_moto_temp = os.path.join(pasta_saida, 'moto_temp')

# Lista de imagens de moto para analisar a cor
imagens_moto = os.listdir(pasta_moto_temp)
pasta_moto_final = os.path.join(pasta_saida, 'moto')

for nome_arquivo in imagens_moto:
    caminho_arquivo = os.path.join(pasta_moto_temp, nome_arquivo)
    print(f"\nAnalisando cor de: {nome_arquivo}")

    img = cv2.imread(caminho_arquivo)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Converte de BGR (OpenCV) para RGB

    # Re-executa a detecção para obter a bounding box
    results = model(caminho_arquivo, verbose=False)

    try:
        # Pega a maior bounding box de moto na imagem
        maior_box = None
        maior_area = 0
        for r in results:
            for box in r.boxes:
                if model.names[int(box.cls)] == 'motorcycle':
                    area = (box.xyxy[0][2] - box.xyxy[0][0]) * (box.xyxy[0][3] - box.xyxy[0][1])
                    if area > maior_area:
                        maior_area = area
                        maior_box = box.xyxy[0]

        if maior_box is None:
            print(f"--> Não foi possível obter a caixa da moto em {nome_arquivo}. Pulando.")
            continue

        # Recorta a imagem para a área da moto
        x1, y1, x2, y2 = map(int, maior_box)
        moto_recortada = img_rgb[y1:y2, x1:x2]

        # Redimensiona para acelerar o K-Means
        pixels = moto_recortada.reshape(-1, 3)

        # Usa K-Means para encontrar a cor dominante (ex: 5 clusters)
        kmeans = KMeans(n_clusters=5, n_init=10, random_state=0)
        kmeans.fit(pixels)

        # Cria um histograma para encontrar o cluster mais comum
        unique, counts = np.unique(kmeans.labels_, return_counts=True)
        # O centro do cluster mais comum é a nossa cor dominante
        cor_dominante_rgb = kmeans.cluster_centers_[unique[counts.argmax()]].astype(int)

        # Obtém o nome da cor mais próxima
        nome_cor = get_color_name(tuple(cor_dominante_rgb))
        print(f"--> Cor dominante detectada: {nome_cor} (RGB: {cor_dominante_rgb})")

        # Cria a pasta da cor se não existir
        pasta_cor_destino = os.path.join(pasta_moto_final, nome_cor)
        os.makedirs(pasta_cor_destino, exist_ok=True)

        # Move a imagem para a pasta final
        shutil.move(caminho_arquivo, os.path.join(pasta_cor_destino, nome_arquivo))
        print(f"--> Imagem movida para: {pasta_cor_destino}")

    except Exception as e:
        print(f"Ocorreu um erro ao processar {nome_arquivo}: {e}")

# Remove a pasta temporária
shutil.rmtree(pasta_moto_temp)
print("\nAnálise de cor e organização final concluídas!")

In [ ]:
# Célula 5: Exibir a estrutura de pastas final
!ls -R {pasta_saida}